# Error Correction in Reads

## Background Info

We can assemble a **genome** from a collection of **reads**. Even though genome sequencing was a breakthrough technology, sequencing machines that identify reads still produce errors a substantial percentage of the time, and these errors are unpredictable. It's difficult to determine if the machine has made an error, and where in the read the error has occurred. Therefore, error correction in reads is a vital first step in genome assembly.

* **Genome**: The collection of an organism's DNA taken from all its chromosomes.
* **Reads**: A substring of a genome used for genome assembly.

## Problem

As is the case with point mutations, the most common type of sequencing error occurs when a single nucleotide frmo a read is interpreted incorrectly.

**Given**: A collection of up to 1000 reads of equal length in FASTA format. Some of these reads were generated with a single-nucleotide error. For each read $s$ in the dataset, one of the following applies:

* $s$ was correctly sequenced and appears in the dataset at least twice (possibly as a reverse complement).
* $s$ is incorrect, it appears in the dataset exactly once, and its Hamming distance is 1 with respect to exactly one correct read in the dataset (or its reverse complement).

**Return**: A list of all corrections in the form "[old read] --> [new read]". (Each correction must be a single symbol subsitution).

## Solution Explaination

In [1]:
# need a function to calculate hamming distance (if 1--> return as the list of old --> new read)
# need a function to check if there's a reverse complement in the dataset that matches s.

In [ ]:
f = open('../Sample_Error_Correction_in_Reads.txt', 'r')
